# Материалы

- [Reuven M. Lerner - Generators, coroutines, and nanoservices || PyCon Africa 2020](https://youtu.be/tkoaeVS2zRQ)
- [Curious Course on Coroutines and Concurrency (David Beazley)](https://youtu.be/Z_OAlIhXziw)
- http://dabeaz.com/coroutines/Coroutines.pdf
- https://stackoverflow.com/questions/9708902/in-practice-what-are-the-main-uses-for-the-yield-from-syntax-in-python-3-3
- https://stackoverflow.com/questions/35518986/whats-the-difference-between-yield-from-and-yield-in-python-3-3-2

## Функция

In [1]:
def myfunc():
    return 1
    return 2
    return 3

In [2]:
import dis

# Even Python compiler ignores return 2 return 3
dis.dis(myfunc)  # => bytecodes from our function

  2           0 LOAD_CONST               1 (1)
              2 RETURN_VALUE


## Генератор (generator)

`yield` turns function into `generator function`. When Python compiles your function it notices `yield` and tags the function as a `generator function`.

generator_function() => generator (object)

In [3]:
def mygen():
    yield 1
    yield 2
    yield 3

In [4]:
mygen()

<generator object mygen at 0x7f70c6e6e810>

In [5]:
dis.show_code(mygen)  # Notice the flag GENERATOR!

Name:              mygen
Filename:          /tmp/ipykernel_91732/1466075880.py
Argument count:    0
Positional-only arguments: 0
Kw-only arguments: 0
Number of locals:  0
Stack size:        1
Flags:             OPTIMIZED, NEWLOCALS, GENERATOR, NOFREE
Constants:
   0: None
   1: 1
   2: 2
   3: 3


In [6]:
dis.dis(mygen)  # три YIELD_VALUE (нет игнорирования!)

              0 GEN_START                0

  2           2 LOAD_CONST               1 (1)
              4 YIELD_VALUE
              6 POP_TOP

  3           8 LOAD_CONST               2 (2)
             10 YIELD_VALUE
             12 POP_TOP

  4          14 LOAD_CONST               3 (3)
             16 YIELD_VALUE
             18 POP_TOP
             20 LOAD_CONST               0 (None)
             22 RETURN_VALUE


In [7]:
## Iterator protocol

In [8]:
for i in 'abc':
    print(i)

a
b
c


In [9]:
for i in mygen():
    print(i)

1
2
3


In [10]:
g = mygen()
g

<generator object mygen at 0x7f70c6e6ece0>

In [11]:
iter(g)

<generator object mygen at 0x7f70c6e6ece0>

In [12]:
g is iter(g)

True

In [13]:
next(g), next(g), next(g)

(1, 2, 3)

In [14]:
try:
    next(g)
except StopIteration as e:
    print(type(e))

<class 'StopIteration'>


## Корутина

In [15]:
def mygen():
    x = None
    while True:
        x = yield x
        x *= 5

In [16]:
g = mygen()

In [17]:
next(g)

In [18]:
g.send(5)

25

In [19]:
g.send(10)

50

## Мой пример